In [61]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [62]:
from litellm import completion


In [63]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [64]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


In [65]:
# chat = ChatLiteLLM(model="gpt-4o-mini").bind(logprobs=True) 
# This does not work



In [66]:
chat = ChatOpenAI(model="gpt-4o-mini").bind(logprobs=True)
# Only OpenAI works for logprobs

In [67]:
chat


RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002E43E695B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002E43E4B6D20>, root_client=<openai.OpenAI object at 0x000002E43E687170>, root_async_client=<openai.AsyncOpenAI object at 0x000002E43E695B80>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'logprobs': True}, config={}, config_factories=[])

In [68]:
messages = [
    HumanMessage(
        content="what model are you"
    )
]

In [69]:
msg = chat.invoke(messages)


In [70]:
msg.response_metadata["logprobs"]["content"][:5]


[{'token': 'I', 'bytes': [73], 'logprob': -0.038042292, 'top_logprobs': []},
 {'token': ' am',
  'bytes': [32, 97, 109],
  'logprob': -0.038042065,
  'top_logprobs': []},
 {'token': ' based',
  'bytes': [32, 98, 97, 115, 101, 100],
  'logprob': -0.000698496,
  'top_logprobs': []},
 {'token': ' on', 'bytes': [32, 111, 110], 'logprob': 0.0, 'top_logprobs': []},
 {'token': ' Open',
  'bytes': [32, 79, 112, 101, 110],
  'logprob': -0.014165984,
  'top_logprobs': []}]

In [71]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

In [72]:
# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

In [73]:
structured_chat = chat.with_structured_output(Joke)
response = structured_chat.invoke(messages)

In [74]:
response # If I force the llm to return a structured output, it does not return the logprobs


{'setup': 'What kind of model am I?',
 'punchline': "I'm an AI model, but I still can't find my way around a runway!",
 'rating': 6}

In [75]:
msg


AIMessage(content="I am based on OpenAI's GPT-3 model. My training includes a wide range of information up until October 2021, and I can assist with various topics, answer questions, and engage in conversation. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 11, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': {'content': [{'token': 'I', 'bytes': [73], 'logprob': -0.038042292, 'top_logprobs': []}, {'token': ' am', 'bytes': [32, 97, 109], 'logprob': -0.038042065, 'top_logprobs': []}, {'token': ' based', 'bytes': [32, 98, 97, 115, 101, 100], 'logprob': -0.000698496, 'top_logprobs': []}, {'token': ' on', 'byt